In [34]:
import sleap_io as sio

In [31]:
video_path = '21Jan_007.mp4'
video = sio.load_video(video_path)

video_subsample = video[0::16]
video_subsample.shape

(394, 1080, 1920, 3)

In [36]:
from brainglobe_registration.elastix.register import (
    run_registration
)
from brainglobe_registration.utils.utils import open_parameter_file
from skimage.color import rgb2gray
from pathlib import Path

file_path = (
    Path.home() / "NIU_dev" / "bg-elastix"
    / "brainglobe_registration"
    / "parameters"
    / "brainglobe_registration"
    / "affine.txt"
)

parameter_lists = [("affine", open_parameter_file(file_path))]
parameter_lists[0][1]["FixedInternalImagePixelType"] = ["float"]
parameter_lists[0][1]["MovingInternalImagePixelType"] = ["float"]

In [43]:
import pandas as pd
import re

for i in range(1, video_subsample.shape[0]):
    fixed_gray = rgb2gray(video_subsample[i-1])
    moving_gray = rgb2gray(video_subsample[i])
    parameters = run_registration(
        fixed_gray,
        moving_gray,
        parameter_lists,
        filter_images=False
    )


    # Regular expression to find the TransformParameters line
    pattern = r'\(TransformParameters ([\d\.\-e ]+)\)'
    input_string = str(parameters)
    # Search for the pattern in the input string
    match = re.search(pattern, input_string)

    if match:
        # Extract the numbers and convert them to floats
        transform_parameters = list(map(float, match.group(1).split()))
        print(transform_parameters)
        with open(f"out.csv", "a") as f:
            f.write(",".join(map(str, transform_parameters)))
            f.write("\n")
    else:
        print("TransformParameters not found")



[1.00392, 0.00170326, 0.000802939, 1.00387, 21.5267, 6.6359]
[1.00309, 0.00154053, 4.5189e-05, 1.00257, 23.0616, 7.02023]
[1.00018, 0.00095347, -0.00031743, 0.999952, 13.3372, 5.02389]
[0.997839, 0.000373893, -0.000339569, 0.997776, 3.66649, 2.55494]
[0.996501, -0.000286266, 5.62813e-05, 0.996817, -0.912539, 1.82742]
[0.997174, -0.000170052, 0.000111219, 0.997268, -1.73865, 1.68325]
[0.997477, -0.000566044, 0.000440244, 0.997622, -2.03673, 1.59549]
[0.999573, -0.000363407, 0.000292235, 0.999652, -2.02303, 0.162071]
[1.00097, -0.000369919, 0.000187309, 1.00091, -1.93614, -0.343806]
[1.00115, -0.000785245, 0.000544394, 1.0012, -2.25704, -0.248522]
[1.00119, -0.00205117, 0.00219359, 1.00075, -2.14663, -4.00891]
[1.00106, -0.00101914, 0.00110898, 1.00001, 0.925326, -11.2511]
[1.00019, -0.000238017, -1.57805e-05, 0.998651, 3.44206, -16.0075]
[1.00072, 0.000405553, 0.000237787, 0.999095, 9.84608, -19.9456]
[1.00132, 0.0011879, -0.000127587, 0.999574, 17.3883, -18.028]
[1.00241, 0.00155584, 6

In [30]:
str(parameters)

'ParameterObject (0x600000b0c8c0)\n  RTTI typeinfo:   elastix::ParameterObject\n  Reference Count: 1\n  Modified Time: 719695\n  Debug: Off\n  Object Name: \n  Observers: \n    none\nParameterMap 0: \n  (CenterOfRotationPoint 959.5 539.5)\n  (CompressResultImage "false")\n  (DefaultPixelValue 0)\n  (Direction 1 0 0 1)\n  (FinalBSplineInterpolationOrder 3)\n  (FixedImageDimension 2)\n  (FixedInternalImagePixelType "float")\n  (HowToCombineTransforms "Compose")\n  (Index 0 0)\n  (InitialTransformParameterFileName "NoInitialTransform")\n  (MovingImageDimension 2)\n  (MovingInternalImagePixelType "float")\n  (NumberOfParameters 6)\n  (Origin 0 0)\n  (ResampleInterpolator "FinalBSplineInterpolator")\n  (Resampler "DefaultResampler")\n  (ResultImageFormat "tiff")\n  (ResultImagePixelType "double")\n  (Size 1920 1080)\n  (Spacing 1 1)\n  (Transform "AffineTransform")\n  (TransformParameters 1.00169 -0.0156542 0.000866974 1.01169 40.3212 14.9217)\n  (UseDirectionCosines "true")\n'

In [44]:
df = pd.read_csv("out.csv")

In [45]:
df.head()

,r11,r21,r12,r22,tx,ty
0,1.003920,0.001703,0.000803,1.003870,21.526700,6.63590
1,1.003090,0.001541,0.000045,1.002570,23.061600,7.02023
2,1.000180,0.000953,-0.000317,0.999952,13.337200,5.02389
3,0.997839,0.000374,-0.000340,0.997776,3.666490,2.55494
4,0.996501,-0.000286,0.000056,0.996817,-0.912539,1.82742
